## Chat Models - Tagging Documents

In [ ]:
%pip install langchain langchain_openai nest_asyncio lxml beautifulsoup4 --upgrade

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'API_KEY_HERE'

In [1]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [2]:
from langchain.document_loaders.sitemap import SitemapLoader
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
import pandas as pd

/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
sitemap_loader = SitemapLoader(web_path="https://understandingdata.com/sitemap.xml")
sitemap_loader.requests_per_second = 5
docs = sitemap_loader.load()

Fetching pages: 100%|##########| 101/101 [00:06<00:00, 16.38it/s]


In [4]:
# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string" },
        "aggressiveness": {"type": "integer"},
        "primary_topic": {"type": "string", "description": "The main topic of the document."},
    },
     "required": ["primary_topic", "sentiment", "aggressiveness"],
}

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
chain = create_tagging_chain(schema, llm, output_key='output')

In [7]:
results = []

# Remove the 0:10 to run on all documents:
for doc in docs[0:10]:
    print(doc)
    chain_result = chain.invoke({'input': doc.page_content})
    results.append(chain_result['output'])

page_content='Software Engineer and Data Engineer - James PhoenixMenuAboutProjectsServicesPostsContactGrow Your Marketing Organisation With Custom Technology & Workflows.I'm James, a full stack software developer with specialist skills in data engineering and digital marketing. I work exclusively with marketing organizations to streamline their operations and grow their business.Book a Free ConsultationWhat I doTrusted by the world’s most innovative teamsServicesData EngineeringReact DevelopmentPython Programming DevelopmentPrompt EngineeringWeb ScrapingSaaS ApplicationsTechnology StackI use the most robust and latest technologies to build digital products. We are experts in React.js, Node.js, Python, Postgres and Google Cloud Platform.Our ReviewsSee more reviews hereRhys FisherJames has been a key measurement partner for a number of years. While running a boutique analytics agency, I would bring Just Understanding Data into a number of growth transformation projects at a time, across 

In [6]:
results

[{'sentiment': 'positive',
  'aggressiveness': 0.5,
  'primary_topic': 'software engineering'},
 {'sentiment': 'passionate',
  'aggressiveness': 0,
  'primary_topic': 'innovative solutions'},
 {'sentiment': 'positive', 'aggressiveness': 0.5, 'primary_topic': 'contact'},
 {'sentiment': 'positive',
  'aggressiveness': 0.5,
  'primary_topic': 'Software & Data Engineering Services'},
 {'sentiment': 'positive',
  'aggressiveness': 0.2,
  'primary_topic': 'software and data engineering'},
 {'sentiment': 'positive',
  'aggressiveness': 0.5,
  'primary_topic': 'data engineering'},
 {'sentiment': 'positive',
  'aggressiveness': 0.5,
  'primary_topic': 'data engineering services'},
 {'sentiment': 'positive',
  'aggressiveness': 0.5,
  'primary_topic': 'React Software Development'},
 {'sentiment': 'positive',
  'aggressiveness': 0.5,
  'primary_topic': 'Python software development'},
 {'sentiment': 'positive',
  'aggressiveness': 0.5,
  'primary_topic': 'Python software development'}]

In [35]:
df = pd.DataFrame(results)

In [39]:
docs[0].metadata

{'source': 'https://understandingdata.com/',
 'loc': 'https://understandingdata.com/',
 'lastmod': '2023-09-28T14:18:53.466Z',
 'changefreq': 'monthly',
 'priority': '1.0'}

In [43]:
# Combine the URLs with the results
df['url'] = [doc.metadata['source'] for doc in docs[0:10]]


In [44]:
df

,sentiment,aggressiveness,primary_topic,url
0,positive,0.5,software engineering,https://understandingdata.com/
1,passionate,0.0,innovative solutions,https://understandingdata.com/about/
2,positive,0.5,contact,https://understandingdata.com/contact/
3,positive,0.5,software & data engineering services,https://understandingdata.com/services/
4,positive,0.5,software and data engineering,https://understandingdata.com/projects/
5,positive,0.5,data engineering,https://understandingdata.com/posts/
6,positive,0.5,data engineering services,https://understandingdata.com/services/data-en...
7,positive,0.5,React Software Development,https://understandingdata.com/services/react-s...
8,positive,0.5,Python software development,https://understandingdata.com/services/python-...
9,positive,0.5,Python software development,https://understandingdata.com/services/python-...
